<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231127-%E5%BE%97%E5%88%B0sample_2023%E7%9A%845d-NDVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [37]:
import os
import pandas as pd
import math

In [ ]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/sample_2023")

In [39]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-11-22').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

1245


In [40]:
# 二阶谐波
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

harmonicIndependents = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2'])

def add_sincos(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

harmonicSentinel = dataset.map(add_sincos)

harmonicTrend = harmonicSentinel.select(harmonicIndependents.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents.length(),numY=1))

harmonicTrendCoefficients = harmonicTrend.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents])

def fit(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('fitted'))

fittedHarmonic = harmonicSentinel.map(fit)

print(fittedHarmonic.size().getInfo())

1245


In [41]:
# 5天序列NDVI
interval = 5
increment = 'day'
start = '2022-05-01'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-09-01').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  timeRadians2 = ee.Image(years).multiply(4 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDVI(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('NDVI'))

fitteds2 = s2makeup.map(add_fittedNDVI)

HANTS = fitteds2.select('NDVI')

In [42]:
# NDVI时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI']


In [43]:
image_agrgt_clip = image_agrgt.clip(roi)

In [44]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id','landcover'],scale=10,tileScale=8)

In [45]:
cols = ['landcover','id','0_NDVI','1_NDVI','2_NDVI','3_NDVI','4_NDVI','5_NDVI','6_NDVI','7_NDVI','8_NDVI','9_NDVI','10_NDVI','11_NDVI','12_NDVI','13_NDVI','14_NDVI','15_NDVI','16_NDVI','17_NDVI','18_NDVI','19_NDVI','20_NDVI','21_NDVI','22_NDVI','23_NDVI','24_NDVI']
inds = range(samples_datesets.size().getInfo())

In [46]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,landcover,id,0_NDVI,1_NDVI,2_NDVI,3_NDVI,4_NDVI,5_NDVI,6_NDVI,7_NDVI,...,15_NDVI,16_NDVI,17_NDVI,18_NDVI,19_NDVI,20_NDVI,21_NDVI,22_NDVI,23_NDVI,24_NDVI
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(27),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,360
landcover,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,15.000000,16.000000,17.000000,18.000000
id,1.000000,4.000000,6.000000,7.000000,8.000000,9.000000,10.000000,13.000000,14.000000,15.000000,...,88.000000,107.000000,32.000000,53.000000,100.000000,47.000000,92.000000,96.000000,114.000000,30.000000
0_NDVI,0.386575,0.571343,0.551880,0.470970,0.427335,0.336329,0.421983,0.614099,0.679539,0.403990,...,0.361795,0.443642,0.575208,0.347535,0.584551,0.393132,0.371023,0.463382,0.414846,0.628905
1_NDVI,0.377951,0.569218,0.537553,0.456529,0.413515,0.331527,0.423733,0.597763,0.663470,0.379317,...,0.358734,0.457797,0.597127,0.348732,0.591325,0.390842,0.365010,0.468948,0.408711,0.634132
2_NDVI,0.375463,0.572436,0.529307,0.448670,0.403692,0.330432,0.429280,0.585846,0.649975,0.362212,...,0.359456,0.473327,0.613539,0.351449,0.599325,0.389217,0.364908,0.475772,0.405772,0.640151


In [48]:
df.to_csv('sample_2023-5d时间序列.csv',encoding='utf-8')